In [ ]:
from VectorDataBase import WeaviateDB
# from Saul_LLM_GGUF import Legal_LLM
from Saul_LLM_HF import Legal_LLM

In [ ]:
chat_obj = Legal_LLM()

In [ ]:
class RAGDemo:
    def __init__(self, collection_names=['Uk', 'Whales']):
        """
        Initializes the RAGDemo object.
        
        Args:
            collection_names (list, optional): A list of collection names. Defaults to ['Uk'].
        """
        self.vector_db = WeaviateDB(collection_names=collection_names)
        self.llm = Legal_LLM()

    def add_text(self, collection_name, text, metadata=None):
        """
        Adds text data to a specified collection in the Weaviate database.
        
        Args:
            collection_name (str): The name of the collection in the database.
            text (str): The text data to be added.
            metadata (dict): Additional metadata associated with the text.
        """
        self.vector_db.add_text_to_db(
            collection_name=collection_name,
            text=text,
            metadata=metadata
        )

    def query(self, collection_name, query, k=1):
        """
        Performs a RAG query on the specified collection using the Ollama LLM.
        
        Args:
            collection_name (str): The name of the collection in the database.
            query (str): The query to search for similar documents.
            k (int, optional): The number of documents to return. Defaults to 1.
        
        Returns:
            None
        
        Prints the similarity score and the content of the top k documents that match the query.
        """
        current_db = self.vector_db.vector_stores[collection_name]
        
        # Create a retriever for the current database
        retriever = current_db.as_retriever(
            search_kwargs={"k": k})

        # Function to format documents into a single context string
        def format_docs(docs):
            print(f'The retrieved documents are:')
            for idx,doc in enumerate(docs):
                print(f'{idx} - Content: {doc.page_content[:50]}...')
            print()
            return "\n\n".join(doc.page_content for doc in docs)
        
        retrieved_docs = retriever.get_relevant_documents(query)
        context = format_docs(retrieved_docs)
        
        response = self.llm.chat(context={context},
                                 query={query}, max_new_tokens=250)
        print('-')
        print(response)

In [ ]:
app = RAGDemo(
    ['Uk', 'Wales']
)

In [ ]:
app.vector_db.validate_collection()

In [ ]:
app.vector_db.delete_collection('Whales')
app.vector_db.delete_collection('Uk')

In [ ]:
app.add_text(
    collection_name='Uk',
    text='''Child labor is a grave social issue that deprives children of their childhood, 
    education, and the opportunity for a better future. It often involves hazardous work that 
    poses significant risks to their physical and mental health. Despite global efforts to eradicate it, 
    millions of children worldwide are still engaged in labor due to poverty, lack of access to education, 
    and inadequate enforcement of labor laws. Ending child labor requires a multifaceted approach, including 
    strengthening legal frameworks, improving educational opportunities, and addressing the root causes of poverty and inequality.''',
)

In [ ]:
# Perform a RAG query with a sample query
app.query(
    collection_name='Uk',
    query="What are the primary factors contributing to the persistence of child labor globally?",
)

In [ ]:
# Perform a RAG query with a sample query
app.query(
    collection_name='Uk',
    query="How can improving educational opportunities help in the eradication of child labor?",
)

In [ ]:
# Perform a RAG query with a sample query
app.query(
    collection_name='Uk',
    query="What measures can governments and organizations take to strengthen legal frameworks against child labor?",
)